In [1]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In this one we'll extend the original code and integrate AI-Agents and self reasoning block.
We are using langGraph for better code orchestration.

In [2]:
from langchain.document_loaders import TextLoader
import os

# Load documents
documents = []
data_dir = "data"
for filename in os.listdir(data_dir):
    if filename.endswith(".txt"):
        loader = TextLoader(os.path.join(data_dir, filename))
        documents.extend(loader.load())

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Chunking
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents(documents)

In [4]:
from langchain.embeddings import HuggingFaceEmbeddings

# Embedding
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


/var/folders/cr/zw_bcxpn257g4khbkhrtn_vm0000gn/T/ipykernel_29274/2890209881.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [5]:
from langchain.vectorstores import FAISS

vectorstore = FAISS.from_documents(chunks, embedding=embedding_model)
retriever = vectorstore.as_retriever()

In [6]:
from langchain.llms import Ollama

# LLM initialization
llm = Ollama(model="gemma3")


/var/folders/cr/zw_bcxpn257g4khbkhrtn_vm0000gn/T/ipykernel_29274/807365898.py:4: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="gemma3")


In [7]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import create_history_aware_retriever

# History-aware retriever
retriever_prompt = (
    "Given a chat history and the latest user question which might reference context in the chat history, "
    "formulate a standalone question which can be understood without the chat history. "
    "Do NOT answer the question, just reformulate it if needed and otherwise return it as is."
)
contextualize_q_prompt = ChatPromptTemplate.from_messages([
    ("system", retriever_prompt),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}")
])
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

LangGraph

In [8]:

from langchain.schema import BaseMessage
from typing import TypedDict, List

# Define the shared state for LangGraph
class GraphState(TypedDict):
    query: str
    chat_history: List[BaseMessage]
    documents: List
    reasoning: str
    answer: str

In [9]:
# Input node
def input_node(state: GraphState) -> GraphState:
    return state


In [10]:
# Retrieval node
def retrieve_node(state: GraphState) -> GraphState:
    question = state['query']
    history = state['chat_history']
    standalone_question = history_aware_retriever.invoke({"chat_history": history, "input": question})
    state["documents"] = standalone_question
    return state

In [11]:
from langchain_core.prompts import PromptTemplate
# Self-reasoning node
def self_reasoning_node(state: GraphState) -> GraphState:
    reasoning_prompt = PromptTemplate.from_template(
        "Given the question: {query}\nand retrieved docs: {docs}\nWhat is a step-by-step reasoning path?"
    )
    reasoning_chain = reasoning_prompt | llm
    reasoning = reasoning_chain.invoke({"query": state["query"], "docs": state["documents"]})
    state["reasoning"] = reasoning
    return state

In [12]:
# Answer generation node
def generate_answer_node(state: GraphState) -> GraphState:
    full_context = f"{state['reasoning']}\n\n{state['documents']}"
    final_prompt = PromptTemplate.from_template(
        "Answer the question: {query}\nUse context:\n{context}"
    )
    answer_chain = final_prompt | llm
    answer = answer_chain.invoke({"query": state["query"], "context": full_context})
    state["answer"] = answer
    return state

In [13]:

# Output node
def output_node(state: GraphState) -> GraphState:
    return state

In [14]:
from langgraph.graph import StateGraph

# Build the LangGraph
graph = StateGraph(GraphState)
graph.add_node("input", input_node)
graph.add_node("retrieve", retrieve_node)
graph.add_node("reason", self_reasoning_node)
graph.add_node("generate", generate_answer_node)
graph.add_node("output", output_node)

graph.set_entry_point("input")
graph.add_edge("input", "retrieve")
graph.add_edge("retrieve", "reason")
graph.add_edge("reason", "generate")
graph.add_edge("generate", "output")
graph.set_finish_point("output")

In [15]:
# Compile app
app = graph.compile()

In [16]:
# Example invocation
example_query = "Summarize the main points of the document."
example_history = []  # Fill with actual history if available

result = app.invoke({
    "query": example_query,
    "chat_history": example_history
})

print("Final Answer:", result["answer"])

Final Answer: Here’s a summary of the main points of the documents:

The documents explore Artificial Intelligence (AI), which encompasses a broad range of intelligent machine tasks. AI is categorized into several types:

*   **Narrow AI:** Specialized systems like chatbots and recommendation engines.
*   **General AI:** Hypothetical AI with human-level intelligence.
*   **Super AI:** AI exceeding human intelligence – a theoretical concept.

Natural Language Processing (NLP) is a key subfield focused on enabling machines to understand and generate human language, exemplified by technologies like Siri and Alexa.

However, the development and use of AI raises significant ethical concerns, including bias, job displacement, privacy, and accountability.  Furthermore, the need for Explainable AI (XAI) is highlighted to promote transparency and trust in AI systems by providing understandable justifications for their decisions.


In [17]:
# Example invocation
example_query = "rephrase previous answer in 100 characters."
result = app.invoke({
    "query": example_query,
    "chat_history": example_history
})

print("Final Answer:", result["answer"])

Final Answer: AI ethics: bias, job loss, privacy, misuse. Trust & accountability vital. (99 characters)
